## Graph data structure
## Copyright: Jagadeesh Vasudevamurthy
## filename:graph.ipynb¶

# All import here

In [1]:
import sys # For getting Python Version
import os
import enum # For enum
import math  # for infinity
from graphviz import Source
print("Version of Python I am using is", sys.version)

Version of Python I am using is 3.9.6 (default, Feb  3 2024, 15:58:27) 
[Clang 15.0.0 (clang-1500.3.9.4)]


# Graph input and output directory setup
# YOU MUST CHANGE 2 lines below

In [2]:
# Base directory paths for input and output files
inputFileBase = "/Users/jilsoncorreia/Downloads/PSA ASS6/assignment7/graphdata/"
outputFileBase = "/Users/jilsoncorreia/Downloads/PSA ASS6/assignment7/graphoutput/" 

In [3]:
def read_dot_file(f:'string')->'dot_graph':
    filename = outputFileBase + f + ".dot"
    print(filename)
    with open(filename) as f1:
        dot_graph = f1.read()
    print(dot_graph)
    return(dot_graph)
# Function to read a DOT file and return its content

# YOU WRITE CODE HERE

# Graph buider from file

In [4]:
############################################################
# GraphBuilder.py
# Author: Jagadeesh Vasudevamurthy
# Copyright: Jagadeesh Vasudevamurthy 2023
###########################################################

############################################################
# YOU WRITE CODE IN THIS FILE
###########################################################

class GraphBuilder:
    def __init__(self, g: "graph", f: "string"):
        self._g = g
        # graph object
        self._f = f  # File from which you are building graph
        self._g._g = self._build_graph()

    ############################################################
    # Write code: build_graph
    # Use as many private functions and prvate data you want
    ###########################################################
    def _build_graph(self) -> "None":
        notReadline = 0
        readline = 0
        # Counter for lines not read and read successfully
        with open(self._f, "r") as file:
            data = file.readlines()
            for aline in data:
                token = aline.split()
                size = len(token)
                if (size < 2) or (size > 3):
                    notReadline = notReadline + 1
                    print("NOT READ LINE", aline)
                    continue
                readline = readline + 1
                tf = token[0]
                tt = token[1]
                if (tf == tt):
                  notReadline = notReadline + 1
                  print("NOT READ LINE", aline)
                  continue ;
                weight = 0
                if size == 3:
                    tw = token[2]
                    weight = float(tw)
                self._g.add_edge(tf, tt, weight)


# Write Graph as a dot file

In [5]:
############################################################
# GraphDot.py
# Author: Jagadeesh Vasudevamurthy
# Copyright: Jagadeesh Vasudevamurthy 2020
###########################################################

############################################################
# YOU WRITE CODE IN THIS FILE
###########################################################

class GraphDot:
    def __init__(self, g, f):
        self._g = g  # Handle to graph
        self._f = f  # File where you write graph in dot format
        self._of = open(self._f, "w")
        self._write_dot()
        self._of.close()

    ############################################################
    # Write code: _write_dot
    # Use as many private functions and prvate data you want
    ###########################################################
    def _write_dot(self):
        self._of.write("## Correia Jilson ####\n")
        self._of.write("digraph g {\n")
        t = self._g.get_graph_type()
        if t == GraphType.UNDIRECTED or t == GraphType.WEIGHTED_UNDIRECTED:
            self._of.write("\t edge [dir=none, color=red]")
        else:
            self._of.write("edge [color=red]")
        self._of.write("\n")
        list_of_nodes = self._g.list_of_nodes()
        for n in list_of_nodes:
            first = n.get_name()
            fanouts_of_n = n.fanout_of_a_node()
            for nf in fanouts_of_n:
                second = nf.get_name()
                w = n.node_get_fanout_node_weight(nf)
                s = ""
                if (t == GraphType.WEIGHTED_DIRECTED or t == GraphType.WEIGHTED_UNDIRECTED):
                    if (t == GraphType.WEIGHTED_DIRECTED or first < second):
                        s = s + " " + first + " -> " + second + " [label = " + str(w) + "]"
                elif (t == GraphType.DIRECTED or first < second):
                    s = s + " " + first + " -> " + second
                if s != "":
                    self._of.write("\t")
                    self._of.write(s)
                    self._of.write("\n")
        self._of.write("}")


# DFS USING TIME STAMP

In [6]:
############################################################
# GraphDfs.py
# Author: Jagadeesh Vasudevamurthy
# Copyright: Jagadeesh Vasudevamurthy 2023
###########################################################

# Timestamp class definition
class Timestamp:
    def __init__(self):
        self.in_time = 0
        self.out_time = 0
############################################################
# Depth First Search using Timestamp
###########################################################
class GraphDfsUsingTimeStamp:
    def __init__(self, graph, graph_name, dfs_order, has_loop, work, dfs_traversal_output_file):
        ## Initialization
        self._graph = graph
        self._graph_name = graph_name
        self._dfs_order = dfs_order
        self._has_loop = has_loop
        self._has_loop[0] = False
        self._work = work
        self._work[0] = 0
        self._dfs_traversal_output_file = dfs_traversal_output_file

        ## Private variables
        self._counter = 0
        self._timestamps = {}
        self._unvisited_nodes = set()
        
        self._perform_dfs()
        self._create_dot_file()

    ##########################################################
    # Generate dot file of DFS traversal
    ##########################################################
    def _create_dot_file(self):
        with open(self._dfs_traversal_output_file, "w") as output_file:
            output_file.write("## Jilson Correia ####\n")
            output_file.write("digraph g {\n")
            output_file.write('\tlabel = "[')  # Write label
            for node in self._dfs_order:
                node_name = node.get_name()
                output_file.write(node_name + " ")
            if self._has_loop[0]:
                output_file.write('] LOOP"\n')
            else:
                output_file.write('] NOLOOP"\n')
            
            for node in self._dfs_order:  # Write in/out times
                node_name = node.get_name()
                output_file.write(f"\t{node_name} [label = <{node_name}<BR /><FONT POINT-SIZE='10'>{self._timestamps[node].in_time}/{self._timestamps[node].out_time}</FONT>>]\n")
            
            graph_type = self._graph.get_graph_type()  # Write edges
            if graph_type in (GraphType.UNDIRECTED, GraphType.WEIGHTED_UNDIRECTED):
                output_file.write("\tedge [dir=none, color=red]\n")
            else:
                output_file.write("\tedge [color=red]\n")
            
            for node in self._graph.list_of_nodes():  # Time complexity: THETA(V + E)
                node_name = node.get_name()
                for fanout_node in node.fanout_of_a_node():
                    fanout_name = fanout_node.get_name()
                    weight = node.node_get_fanout_node_weight(fanout_node)
                    edge_desc = ""
                    if graph_type in (GraphType.WEIGHTED_UNDIRECTED, GraphType.WEIGHTED_DIRECTED):
                        if graph_type == GraphType.WEIGHTED_DIRECTED or (node_name < fanout_name):
                            edge_desc = f"  {node_name} -> {fanout_name} [label = {weight}]"
                    elif graph_type == GraphType.DIRECTED or (node_name < fanout_name):
                        edge_desc = f"  {node_name} -> {fanout_name}"
                    if edge_desc:
                        output_file.write(f"\t{edge_desc}\n")
            output_file.write("}")

    def _initialize_data(self):
        for node in self._graph.list_of_nodes():
            self._timestamps[node] = Timestamp()
            self._unvisited_nodes.add(node)
        num_vertices = self._graph.get_numV()
        assert num_vertices == len(self._timestamps)
        assert num_vertices == len(self._unvisited_nodes)
    
    ##########################################################
    # Check if a node is visited
    ##########################################################
    def _is_node_visited(self, node):
        return node not in self._unvisited_nodes
    
    ##########################################################
    # Mark a node as visited
    ##########################################################
    def _mark_node_as_visited(self, node):
        self._unvisited_nodes.remove(node)
        
    ##########################################################
    # Get the first unvisited node
    ##########################################################
    def _get_first_unvisited_node(self):
        return next(iter(self._unvisited_nodes), None)
    
    ##########################################################
    # Recursive DFS implementation
    ##########################################################
    def _recursive_dfs(self, node, parent_node):
        self._work[0] += 1
        if not self._is_node_visited(node):
            self._mark_node_as_visited(node)
            self._counter += 1
            self._timestamps[node].in_time = self._counter
            for neighbor in node.fanout_of_a_node():
                if neighbor != parent_node:
                    self._recursive_dfs(neighbor, node)
            self._counter += 1
            self._timestamps[node].out_time = self._counter
            self._dfs_order.append(node)
        else:
            if self._timestamps[node].out_time == 0:  # Node re-entered before exiting
                self._has_loop[0] = True
                
    ##########################################################
    # Perform DFS on the graph
    ##########################################################
    def _perform_dfs(self):
        self._initialize_data()
        while True:
            node = self._get_first_unvisited_node()
            if node:
                self._recursive_dfs(node, node)
            else:
                break
        self._dfs_order.reverse()


# Dijkstra Algorithm

In [7]:
############################################################
# GraphDijkstra.py
# Author: Jagadeesh Vasudevamurthy
# Copyright: Jagadeesh Vasudevamurthy 2023
###########################################################

############################################################
# All imports
###########################################################
import heapq

############################################################
# GraphDijkstra class definition
###########################################################
class GraphDijkstra:
    def __init__(
        self,
        g: "graph",
        graph_name: "string",
        start_city_name: "string",
        cost: "list of Nodes",  # FILL: cost from start city to all other city. If not reachable -1
        work: "list of size 1",  # FILL
        dijkstra_traversal_dot_output_file: "String",
        show: "bool",
    ):
        ##NOTHING CAN BE CHANGED HERE
        self._g = g
        self._gname = graph_name
        self._start_city_name = start_city_name
        self._cost = cost
        self._work = work
        self._work[0] = 0
        self._dijkstra_traversal_dot_output_file = dijkstra_traversal_dot_output_file
        self._show = show

        ##YOU CAN has any number of private varibles and funcions

        ## Private variables
        self._Dijkstra()
        # self._write_dot()

    ##########################################################
    # All private functions below
    ##########################################################
    def _increment_work(self, count):
        self._work[0] += count

    ##########################################################
    # Dijkstra algorithm
    ##########################################################
    def _Dijkstra(self):
        start = self._start_city_name
        priority_queue = [(0, start)]  # (cost, node)
        node_list = self._g.list_of_nodes()
        node_indices = {node.get_name(): idx for idx, node in enumerate(node_list)}
        self._cost[:] = [float('inf')] * self._g.get_numV()
        self._cost[node_indices[start]] = 0
        visited = set()

        while priority_queue:
            current_node_cost, current_node_name = heapq.heappop(priority_queue)
            current_node_index = node_indices[current_node_name]
            if current_node_name in visited:
                continue
            visited.add(current_node_name)

            current_node = node_list[current_node_index]

            for neighbor in current_node.fanout_of_a_node():
                neighbor_name = neighbor.get_name()
                neighbor_index = node_indices[neighbor_name]
                edge_weight = current_node.node_get_fanout_node_weight(neighbor)
                new_cost_node = current_node_cost + edge_weight

                if new_cost_node < self._cost[neighbor_index]:
                    self._cost[neighbor_index] = new_cost_node
                    heapq.heappush(priority_queue, (new_cost_node, neighbor_name))
                    self._increment_work(1)

        # Filling the cost list with -1 where the node ia unreachable
        for i in range(len(self._cost)):
            if self._cost[i] == float('inf'):
                self._cost[i] = -1

        if self._show:
            print(f"Costs from {start}: {self._cost}")


# BFS

In [8]:
############################################################
# GraphBfs.py
# Author: Jagadeesh Vasudevamurthy
# Copyright: Jagadeesh Vasudevamurthy 2024
###########################################################

############################################################
# Breadth First search
###########################################################
class GraphBfs:
    def __init__(
        self,
        g: "graph",
        graph_name: "string",
        start_city_name: "string",
        end_city:"string",
        shortest_dist:'list of size 1',
        work: "list of size 1",
        bfs_traversal_output_file:"String",
        show: 'bool'
    ):
        ##NOTHING CAN BE CHANGED HERE
        self._g = g
        self._graph_name = graph_name
        self._start_city_name = start_city_name
        self._end_city = end_city
        self._shortest_dist = shortest_dist
        self._shortest_dist[0] = 0
        self._work = work
        self._work[0] = 0
        self._bfs_traversal_output_file = bfs_traversal_output_file
        self._show = show
        ## You can add any private data structures here
        self._a = {}  ## parallel dictionary. Key is node. Value is NodeData
        self._dq = Deque();
        self._bfsorder= []
        self._bfspath= {} #Dictionary. Key is node. Value is from(node)
        
        ## must write 3 routines
        self._bfs() 
        self._find_shortest_path()
        self._write_dot()
        
    ##########################################################
    # Write dot file of BFS traversal
    ##########################################################
    def _write_dot(self):
        with open(self._bfs_traversal_output_file, "w") as f:
            f.write("## Correia Jilson ####\n")
            f.write("digraph g {\n")

            for node in self._bfsorder:
                f.write(f'  "{node}" [label="{node}"]\n')

            for node in self._bfspath:
                from_node = self._bfspath[node]
                if from_node:
                    f.write(f'  "{from_node}" -> "{node}"\n')

            f.write("}\n")


    ##########################################################
    # BFS
    #   TIME: THETA(V)
    #   SPACE:THETA(V)
    ##########################################################
    def _bfs(self):
        start = self._start_city_name
        self._dq.append(start)
        self._visited[start] = True
        self._bfspath[start] = None

        while self._dq:
            current_node = self._dq.popleft()
            self._bfsorder.append(current_node)

            for neighbor in self._g.get_edges(current_node):
                if neighbor not in self._visited:
                    self._visited[neighbor] = True
                    self._bfspath[neighbor] = current_node
                    self._dq.append(neighbor)
                    self._increment_work(1)

  
    ##########################################################
    # Find shortest path from start city to every other city
    ##########################################################
    def _find_shortest_path(self)->'None':
        pend = self._end_city
        if end not in self._visited:
            self._shortest_dist[0] = -1
            return

        path = []
        current_node = end
        while current_node is not None:
            path.append(current_node)
            current_node = self._bfspath.get(current_node)
        path.reverse()

        self._shortest_dist[0] = len(path) - 1

        if self._show:
            print(f"Shortest path from {self._start_city_name} to {self._end_city}: {' -> '.join(path)}")
            print(f"Distance: {self._shortest_dist[0]}")

    def _increment_work(self, c):
        self._work[0] = self._work[0] + c



## NOTHING CAN BE CHANGED BELOW

# Graph Data

In [9]:
###########################################################
# Data.py
# Author: Jagadeesh Vasudevamurthy
# Copyright: Jagadeesh Vasudevamurthy 2023
###########################################################

############################################################
# All imports
###########################################################


class Data:
    def __init__(self, n: "string"):
        self._name = n  ### _name is used as key for this object
        self.age = 100  ## To show you can have anything,

    def __hash__(self):
        t = hash(self._name)
        return t

    def __eq__(self, other: "Node") -> "bool":
        if not isinstance(other, type(self)):
            assert False
        return self._name == other._name

    def __str__(self):
        return self._name

    def get_key(self) -> "string":
        return self._name


## Graph Types

In [10]:
class GraphType(enum.Enum): 
    NONE = 0
    UNDIRECTED = 1
    DIRECTED = 2
    WEIGHTED_UNDIRECTED = 3
    WEIGHTED_DIRECTED  = 4

## Graph class

In [11]:
############################################################
# Graph.py
# Author: Jagadeesh Vasudevamurthy
# Copyright: Jagadeesh Vasudevamurthy 2024
###########################################################

############################################################
# NOTHING CAN BE CHANGED IN THIS FILE
###########################################################


############################################################
# Edge
# T can be number or name or user key
# Decision taken on other
# Must write __eq__ and __hash__
###########################################################
class Edge:
    def __init__(self, n: "Node", weight: "float"):
        # All keys below. 
        # Decision is taken based on key. Do not change key after inserted
        self._other = n  # _other is node
        # All values below
        # We can change these values
        self._weight = weight  # _weight is float

    ############################################################
    # All Public routines. YOU SHOULD ONLY CALL THESE ROUTINES
    ###########################################################
    def get_node(self) -> "Node":
        return self._other

    def get_name(self) -> "T":
        return self.get_node().get_name()

    def get_weight(self) -> "float":
        return self._weight

    def change_weight(self, w: "float") -> "None":
        self._weight = w

    ############################################################
    # All Private routines. YOU SHOULD NOT CALL THESE ROUTINES
    ###########################################################

    #########  WRITE 5 below starts ################
    def _get_key(self) -> 'T':
        return self.get_name()

    def __hash__(self) -> 'int':
        return hash(self._get_key())

    def __lt__(self, other: 'Node') -> 'bool':
        return self._get_key() < other._get_key()

    def __eq__(self, other) -> 'bool':
        return not (self < other) and not (other < self)

    def __ne__(self, other) -> 'bool':
        return not self == other
    #########  WRITE 5 below ends ################


############################################################
# Node
# T can be number or name or user key
# Decision taken on T
# Must write __eq__ and __hash__
###########################################################
class Node:
    def __init__(self, T: "T"):
        # node data structures
        # All keys below. 
        # Decision is taken based on key. Do not change key after inserted
        self._item = T  # self._item is our key in hash/set
        # All values below
        # We can change these values
        self._fanins = {}  # dict of fanins of Node. Key is edge other node name, Value is Edge
        self._fanouts = {}  # dict of fanouts of Node. Key is edge other node name , Value is Edge

    ############################################################
    # All Public routines. YOU SHOULD ONLY CALL THESE ROUTINES
    ###########################################################
    def get_name(self) -> "T":
        return self._item

    def add_fan_out(self, e: "Edge") -> "None":
        self._fanouts[e.get_name()] = e

    def add_fan_in(self, e: "Edge") -> "None":
        self._fanins[e.get_name()] = e

    def num_fan_outs(self) -> "int":
        return len(self._fanouts)

    def num_fan_ins(self) -> "int":
        return len(self._fanins)

    def fanout_of_a_node_edges(self) -> "list of fanout edges":
        return list(self._fanouts.values())

    def fanout_of_a_node(self) -> "list of fanout Nodes":
        return [edge.get_node() for edge in self.fanout_of_a_node_edges()]

    def fanin_of_a_node_edges(self) -> "list of fanin edges":
        return list(self._fanins.values())

    def fanin_of_a_node(self) -> "list of fanin Nodes":
        return [edge.get_node() for edge in self.fanin_of_a_node_edges()]

    def node_has_fanout_edge(self, e: "Edge") -> "Edge or None":
        return self._fanouts.get(e.get_name())

    def node_has_fanout(self, e: "Edge") -> "Node or None":
        someedge = self.node_has_fanout_edge(e)
        return someedge.get_node() if someedge else None

    def node_has_fanin_edge(self, e: "Edge") -> "Edge or None":
        return self._fanins.get(e.get_name())

    def node_has_fanin(self, e: "Edge") -> "Node or None":
        someedge = self.node_has_fanin_edge(e)
        return someedge.get_node() if someedge else None

    def node_get_fanout_node_weight(self, other_node: "Node") -> "float":
        someedge = self._fanouts.get(other_node.get_name())
        assert self.get_name() != someedge.get_name()
        return someedge.get_weight()

    def node_get_fanin_node_weight(self, other_node: "Node") -> "float":
        someedge = self._fanins.get(other_node.get_name())
        assert self.get_name() != someedge.get_name()
        return someedge.get_weight()
        

    ############################################################
    # All Private routines. YOU SHOULD NOT CALL THESE ROUTINES
    ###########################################################

    #########  WRITE 5 below starts ################
    def _get_key(self) -> 'T':
        return self.get_name()

    def __hash__(self) -> 'int':
        return hash(self._get_key())

    def __lt__(self, other: 'Node') -> 'bool':
        return self._get_key() < other._get_key()

    def __eq__(self, other) -> 'bool':
        return not (self < other) and not (other < self)

    def __ne__(self, other) -> 'bool':
        return not self == other
    #########  WRITE 5 below ends ################


class Graph:
    ##GRAPH DATA STRUCTURE
    def __init__(self, graphtype: "GraphType"):
        self._numE = 0  # Number of edges
        self._type = graphtype  # Graph type
        self._dict = {}  # Key is Node key, Value is Node

    ############################################################
    # All Public routines. YOU SHOULD ONLY CALL THESE ROUTINES
    ###########################################################
    def is_directed_graph(self) -> "bool":
        return self._type in {GraphType.DIRECTED, GraphType.WEIGHTED_DIRECTED}

    def is_undirected_graph(self) -> "bool":
        return not self.is_directed_graph()
        
    def is_weighted_graph(self) -> "bool":
        return self._type in {GraphType.WEIGHTED_UNDIRECTED, GraphType.WEIGHTED_DIRECTED}
       
    def get_graph_type(self) -> "GraphType":
        return self._type

    def get_graph_type_as_string(self) -> "string":
        t = self.get_graph_type()
        return { GraphType.UNDIRECTED: "UNDIRECTED GRAPH",GraphType.DIRECTED: "DIRECTED GRAPH",GraphType.WEIGHTED_UNDIRECTED: "WEIGHTED_UNDIRECTED_GRAPH",GraphType.WEIGHTED_DIRECTED: "WEIGHTED_DIRECTED_GRAPH",}.get(t, "NONE")

    def get_numV(self) -> "int":
        return len(self._dict)

    def get_numE(self) -> "int":
        return self._numE

    def list_of_nodes(self) -> "list of nodes":
        return list(self._dict.values())

    def build_node(self, name: 'T') -> "Node":
        return Node(name)
    
    def build_node_and_to_graph(self, name: 'T') -> "Node":
        n = Node(name)
        self._add_node(n)
        return n

    def has_node(self, node: 'Node') -> "bool":
        return node in self._dict

    def get_node(self, node: 'Node') -> 'Node':
        return self._dict.get(node)

    def add_edge(self, fname: 'T', tname: 'T', w: 'float'):
        f = Node(fname)
        t = Node(tname)
        self._add_edge(f, t, w)
        
    ##########################################################
    # NOTHING CAN BE CHANGED IN THIS ROUTINE
    # if this passes means all your interface is right
    #  #Use this routine as a pattern for all other routine
    ##########################################################
    def dump(self, name):
        print("------------", name, "------------ ")
        print(self.get_graph_type())
        print("Num Vertices =", self.get_numV())
        print("Num Edges    =", self._numE)
        numedge = 0
        all_nodes = self.list_of_nodes()
        for node in all_nodes:
            n = node.get_name()
            print(n, "Fanouts: ", end="")
            f = node.num_fan_outs()
            if f == 0:
                print("NONE")
            else:
                fanout_nodes = node.fanout_of_a_node()
                for j, fnode in enumerate(fanout_nodes):
                    numedge += 1
                    d = fnode.get_name()
                    end_char = "," if j < f - 1 else "\n"
                    print(d, end=end_char)
            print(n, "Fanins : ", end="")
            f = node.num_fan_ins()
            if f == 0:
                print("NONE")
            else:
                fanin_nodes = node.fanin_of_a_node()
                for j, fnode in enumerate(fanin_nodes):
                    numedge += 1
                    d = fnode.get_name()
                    end_char = "," if j < f - 1 else "\n"
                    print(d, end=end_char)
        assert numedge // 2 == self._numE

    ##########################################################
    # Nothing can be changed
    # TIME: THETA(V + E)
    # SPACE: THETA(V)
    ##########################################################
    def assert_dfs_passed(self, has_loop: "bool", dfs_order: "list of nodes"):
        t = self.get_graph_type()
        if t in {GraphType.UNDIRECTED, GraphType.WEIGHTED_UNDIRECTED}:
            return
        if not has_loop:
            set_of_visited_nodes = set()
            for n in dfs_order:
                ## Go on fanins of node
                for nf in n.fanin_of_a_node():
                    assert nf in set_of_visited_nodes  # find in THETA(1)
                set_of_visited_nodes.add(n)  # add in THETA(1)
            # All nodes must be visited
            assert len(set_of_visited_nodes) == self.get_numV()
            print("DFS ASSERT PASSED")

    ############################################################
    # All Private routines. YOU SHOULD NOT CALL THESE ROUTINES
    ###########################################################
    def _add_node(self, n: 'Node') -> "Node":
        nstored = self.get_node(n)
        if nstored:
            return nstored
        self._dict[n] = n
        nstored = self.get_node(n)
        assert nstored
        return nstored

    def _add_an_edge(self, f: 'Node', t: 'Node', fanout: 'bool', w: 'float'):
        f = self._add_node(f)
        t = self._add_node(t)
        if fanout:
            ed = f.node_has_fanout_edge(t)
            if ed:
                if w < ed.get_weight():
                    ed.change_weight(w)
            else:
                ed = Edge(t, w)
                self._numE += 1
                f.add_fan_out(ed)
        else:
            ed = f.node_has_fanin_edge(t)
            if ed:
                if w < ed.get_weight():
                    ed.change_weight(w)
            else:
                ed = Edge(t, w)
                f.add_fan_in(ed)
        
    ##########################################################
    # Nothing can be changed in _add_edge
    # You must write _add_an_edge for this routine to work
    ##########################################################
    def _add_edge(self, f: 'Node', t: 'Node', w: 'float'):
        self._add_an_edge(f, t, True, w)  # fanout
        self._add_an_edge(t, f, False, w)  # fanin
        if self.is_undirected_graph():
            self._add_an_edge(t, f, True, w)  # fanout
            self._add_an_edge(f, t, False, w)  # fanin


    ############################################################
    ## All the routines written by students
    ##########################################################
    def build_graph(self, f: "file name"):
        b = GraphBuilder(self, f)  

    def write_dot(self, f):
        b = GraphDot(self, f)

    def show_dot_file(self, filename: "string"):
        with open(filename) as f:
            dot_graph = f.read()
        return dot_graph

    def dfs_using_time_stamp(
        self,
        gname: "string",
        dfs_order: "list of nodes",
        has_loop: "List of size 1 Boolean",
        work: "list of size 1",
        dfs_dot_output_file: "Traversal file name",
    ):
        b = GraphDfsUsingTimeStamp(
            self, gname, dfs_order, has_loop, work, dfs_dot_output_file
        )

    def bfs(
        self,
        gname: "graphname",
        start: "start city name",
        endt: "end city name",
        dist: "list of size 1",
        work: "list of size 1",
        ofile: "bfs_traversal_output_file",
        show: "bool",
    ):
        b = GraphBfs(self, gname, start, endt, dist, work, ofile, show)

    def Dijkstra(
        self,
        gname: "string",
        start_city: "String",
        cost: "list of cost",  # Caller will Fill. cost from start city to all other city.If not reachable -1
        work: "list of size 1",
        dijkstra_traversal_dot_output_file: "Traversal file name",
        show: "bool",
    ):
        b = GraphDijkstra(
            self,
            gname,
            start_city,
            cost,
            work,
            dijkstra_traversal_dot_output_file,
            show,
        )


# All testing routines here. Nothing can be changed
# Use this routines to understand how to write testbench

# DFS TESTER

In [12]:
############################################################
# GraphDfsTest.py
# Author: Jagadeesh Vasudevamurthy
# Copyright: Jagadeesh Vasudevamurthy 2023
###########################################################
    
############################################################
# All imports
###########################################################
'''
from GraphType import *  ## #Otherwise, you cannot use GraphType
from Graph import *
from Data import *  ##User UDT
from GraphGlobal import *
'''


class GraphDfsTest:
    def __init__(self):
        self._show = False  # Change to True for debugging
        self._test_one()
        self._test()

    def _test1(self, gname: "graphname", graphtype: "GraphType", expected_has_loop: "Bool"):
        full_name = inputFileBase + gname + ".txt"

        print("Building graph", full_name)
        g = Graph(graphtype)
        g.build_graph(full_name)
        if self._show:
            g.dump(full_name)

        dot_output_file = outputFileBase + gname + ".dot"
        g.write_dot(dot_output_file)

        dfs_order = []  # Caller will Fill. List of Nodes
        has_loop = [False]  # List of size 1
        work = [0]  # List of size 1
        dfs_dot_output_file = outputFileBase + gname + "dfs.dot"
        g.dfs_using_time_stamp(gname, dfs_order, has_loop, work, dfs_dot_output_file)

        print("DFS traversal is in")
        print(dfs_dot_output_file)

        print("DFS ORDER: ", end=" ")
        for node in dfs_order:
            p1 = node.get_name()
            print(p1, end=" ")
        print()

        if has_loop[0]:
            print("LOOP")
        else:
            print("NOLOOP")
        print("Work Done", work[0])
        assert has_loop[0] == expected_has_loop
        g.assert_dfs_passed(has_loop[0],dfs_order)

    def _test_one(self):
        g = [
            ["1", GraphType.UNDIRECTED, False],
        ]

        for g1 in g:
            self._test1(g1[0], g1[1], g1[2])

    def _test(self):
        g = [
            ["u1", GraphType.UNDIRECTED, False],
            ["1", GraphType.UNDIRECTED, False],
            ["udf1", GraphType.DIRECTED, True],
            ["2", GraphType.DIRECTED, False],
            ["3", GraphType.DIRECTED, True],
            ["cat", GraphType.DIRECTED, False],
            ["7", GraphType.DIRECTED, False],
        ]

        for g1 in g:
            self._test1(g1[0], g1[1], g1[2])


# BFS Tester

In [13]:
############################################################
# GraphBfsTest.py
# Author: Jagadeesh Vasudevamurthy
# Copyright: Jagadeesh Vasudevamurthy 2024
###########################################################


class GraphBfsTest:
    def __init__(self):
        self._show = False  # Change to True for debugging
        self._test_one()
        #self._test()

    def _test1(self, gname: "graphname", graphtype: "GraphType", start: "start city name", endt: "end city name",edist:'int',show:'bool'):
        full_name = inputFileBase + gname + ".txt"
        print("Building graph", full_name)
        g = Graph(graphtype)
        g.build_graph(full_name)
        if self._show:
            g.dump(full_name)

        dot_output_file = outputFileBase + gname + ".dot"
        g.write_dot(dot_output_file)
        dist =[0] #Distance from start city to end city
        work = [0]  # List of size 1
        bfs_traversal_output_file = outputFileBase + gname + "bfs.dot"
        g.bfs(gname,start,endt, dist, work,bfs_traversal_output_file, show) ;
        assert(work[0] > 0)
        assert(dist[0] == edist)
        print("Work Done", work[0])
        print("Shortest Distance from",start, "to",endt, "is", dist[0])
        

    def _test_one(self):
        g = [
            ["5",GraphType.UNDIRECTED,"0","6",4,True] #name, directed/undirected, startcity, endcity,expected distance,show
        ]

        for g1 in g:
            self._test1(g1[0], g1[1], g1[2],g1[3],g1[4],g1[5])

    def _test(self):
        pass





# Dijkstra Tester

In [14]:
############################################################
# GraphDijkstraTest.py
# Author: Jagadeesh Vasudevamurthy
# Copyright: Jagadeesh Vasudevamurthy 2023
###########################################################

class GraphGraphDijkstraTest:
    def __init__(self):
        self._show = False  # Change to True for debugging
        self._test()

    def _test(self):
        n = ["7", "loopparallel", "17", "hd1", "hd2", "hd3", "g1", "hd4", "hd5","hd6"]
        t = [
            GraphType.WEIGHTED_DIRECTED,
            GraphType.WEIGHTED_UNDIRECTED,
            GraphType.WEIGHTED_UNDIRECTED,
            GraphType.WEIGHTED_UNDIRECTED,
            GraphType.WEIGHTED_UNDIRECTED,
            GraphType.WEIGHTED_UNDIRECTED,
            GraphType.WEIGHTED_DIRECTED,
            GraphType.WEIGHTED_UNDIRECTED,
            GraphType.WEIGHTED_UNDIRECTED,
            GraphType.WEIGHTED_UNDIRECTED,
        ]
        s = ["0", "s", "A", "17", "60", "85", "P", "49","1","1"]  # starting city
        w0 = [0.0, 5.0, 3.0, 9.0, 13.0, 8.0, 7.0]
        w1 = [0.0, 1.0, 6.0, 7.0]  
        w2 = [2.0, 7.0, 5.0, 1.0, 3.0, 7.0, 0.0]
        w3 = [
            20.0,
            22.0,
            25.0,
            27.0,
            25.0,
            68.0,
            86.0,
            39.0,
            70.0,
            36.0,
            53.0,
            91.0,
            35.0,
            88.0,
            30.0,
            43.0,
            0.0,
            54.0,
            74.0,
            41.0,
        ]
        w4 = [
            9.0,
            13.0,
            8.0,
            10.0,
            8.0,
            5.0,
            8.0,
            5.0,
            12.0,
            1.0,
            7.0,
            15.0,
            4.0,
            8.0,
            9.0,
            4.0,
            11.0,
            1.0,
            4.0,
            12.0,
            9.0,
            11.0,
            7.0,
            9.0,
            10.0,
            9.0,
            7.0,
            10.0,
            5.0,
            10.0,
            11.0,
            9.0,
            1.0,
            7.0,
            12.0,
            6.0,
            12.0,
            15.0,
            10.0,
            11.0,
            15.0,
            6.0,
            10.0,
            7.0,
            9.0,
            7.0,
            7.0,
            14.0,
            5.0,
            13.0,
            8.0,
            8.0,
            10.0,
            7.0,
            4.0,
            6.0,
            3.0,
            8.0,
            11.0,
            11.0,
            12.0,
            4.0,
            9.0,
            9.0,
            7.0,
            7.0,
            7.0,
            0.0,
            13.0,
            6.0,
            7.0,
            8.0,
            8.0,
            3.0,
            5.0,
            6.0,
            11.0,
            5.0,
        ]
        w5 = [
            154.0,
            98.0,
            90.0,
            49.0,
            186.0,
            190.0,
            178.0,
            114.0,
            123.0,
            -1.0,
            -1.0,
            -1.0,
            123.0,
            -1.0,
            104.0,
            -1.0,
            -1.0,
            -1.0,
            207.0,
            134.0,
            123.0,
            75.0,
            155.0,
            -1.0,
            198.0,
            68.0,
            90.0,
            170.0,
            135.0,
            -1.0,
            103.0,
            145.0,
            -1.0,
            54.0,
            111.0,
            163.0,
            173.0,
            115.0,
            87.0,
            159.0,
            -1.0,
            94.0,
            102.0,
            -1.0,
            76.0,
            67.0,
            167.0,
            138.0,
            216.0,
            -1.0,
            172.0,
            102.0,
            212.0,
            163.0,
            103.0,
            112.0,
            -1.0,
            182.0,
            145.0,
            92.0,
            -1.0,
            -1.0,
            194.0,
            -1.0,
            182.0,
            -1.0,
            201.0,
            96.0,
            -1.0,
            85.0,
            121.0,
            108.0,
            161.0,
            130.0,
            100.0,
            120.0,
            -1.0,
            118.0,
            215.0,
            92.0,
            156.0,
            162.0,
            163.0,
            168.0,
            0.0,
            71.0,
            110.0,
            -1.0,
            -1.0,
            190.0,
            217.0,
            100.0,
            105.0,
            178.0,
        ]
        w6 = [0.0, 1.0, 4.0, 4.0, 2.0, 3.0]

        h4 = [13.0, 20.0, 30.0, 17.0, 17.0, 34.0, 33.0, 16.0, 16.0, 9.0, 11.0, 31.0, 21.0, 34.0, 19.0, 14.0, 23.0, 20.0, 42.0, 21.0, 15.0, 19.0, 24.0, 15.0, 34.0, 42.0, 27.0, 0, 10.0, 15.0, 16.0, 18.0, 19.0, 18.0, 14.0, 21.0, 21.0, 28.0, 21.0, 19.0, 45.0, 26.0, 27.0, 20.0, 16.0, 23.0, 28.0, 22.0, 27.0, 18.0, 22.0, 21.0, 11.0, 18.0, 22.0, 14.0, 26.0, 27.0, 27.0, 17.0, 24.0, 17.0, 17.0, 22.0, 12.0, 21.0, 10.0, 25.0]          
        h5 = [0, 8.0, 3.0, 11.0, 6.0, 8.0, 7.0, 6.0, 12.0, 12.0, 10.0, 9.0, 18.0, 12.0, 4.0, 8.0, 1.0, 3.0, 1.0, 2.0, 10.0, 8.0, 5.0, 6.0, 9.0, 17.0, 11.0, 8.0]
        h6 = [0, 6.0, 3.0, 3.0, 4.0, 3.0, 5.0, 3.0, 3.0, 4.0, 6.0, 5.0, 5.0, 4.0, 4.0, 1.0, 4.0, 5.0, 7.0, 6.0, 6.0, 4.0, 4.0, 4.0, 1.0, 4.0, 5.0, 7.0, 5.0, 2.0, 4.0, 5.0, 5.0, 3.0, 5.0, 6.0, 5.0, 4.0, 5.0, 3.0, 5.0, 5.0, 6.0, 4.0, 5.0, 5.0, 3.0, 6.0, 6.0, 5.0, 3.0, 6.0, 7.0, 5.0, 2.0, 6.0, 6.0, 4.0, 4.0, 3.0, 4.0, 5.0, 3.0, 6.0, 5.0, 3.0, 3.0, 4.0, 7.0, 2.0, 4.0, 6.0, 5.0, 2.0, 2.0, 6.0, 4.0, 6.0, 4.0, 5.0, 3.0, 4.0, 4.0, 4.0, 2.0, 1.0, 3.0, 3.0]
        
        w = [w0, w1, w2, w3, w4, w5, w6,h4,h5,h6]

        n1 = len(n)
        t1 = len(t)
        s1 = len(s)
        d1 = len(w)
        assert n1 == t1
        assert n1 == s1
        assert n1 == d1
        for i in range(n1):
            gname = n[i]
            start_city = s[i]
            expected_cost_array = w[i]
            print("------------- ", gname, " ------------")
            full_name = inputFileBase + gname + ".txt"
            g = Graph(t[i])
            g.build_graph(full_name)
            if self._show:
                g.dump(full_name)
                dot_output_file = outputFileBase + gname + ".dot"
                g.write_dot(dot_output_file)

            cost = (
                []
            )  # Caller will Fill. cost from start city to all other city.If not reachable -1
            work = [0]  # List of size 1
            dijkstra_traversal_dot_output_file = outputFileBase + gname + "dijkstra.dot"
            show = False
            if g.get_numV() < 21:
                show = True
            g.Dijkstra(
                gname, start_city, cost, work, dijkstra_traversal_dot_output_file, show
            )
            # assert answers
            print("Expected cost = ", expected_cost_array)
            print("Your answer    = ", cost)
            assert len(cost) == len(expected_cost_array)
            x = 0
            for j in range(len(cost)):
                if cost[j] != expected_cost_array[j]:
                    x = x + 1
            print("Failed ", x)
            assert x == 0  # NO failure can happen


# Graph Representation Tester

In [15]:
class GraphRepresentationTest:
    def __init__(self):
        self._test()

    def _u1(self):
        name = "13"
        f = inputFileBase + name + ".txt"
        g = Graph(GraphType.UNDIRECTED)
        g.build_graph(f)
        g.dump(name)
        file = outputFileBase + name + ".dot"
        g.write_dot(file)
        Source(read_dot_file(name))
        assert g.get_numV() == 7
        assert g.get_numE() == 24

    def _uw1(self):
        name = "14"
        f = inputFileBase + name + ".txt"
        g = Graph(GraphType.WEIGHTED_UNDIRECTED)
        g.build_graph(f)
        g.dump(name)
        file = outputFileBase + name + ".dot"
        g.write_dot(file)
        Source(read_dot_file(name))
        assert g.get_numV() == 6
        assert g.get_numE() == 20

    def _d1(self):
        name = "15"
        f = inputFileBase + name + ".txt"
        g = Graph(GraphType.DIRECTED)
        g.build_graph(f)
        g.dump(name)
        file = outputFileBase + name + ".dot"
        g.write_dot(file)
        Source(read_dot_file(name))
        assert g.get_numV() == 6
        assert g.get_numE() == 6

    def _dw1(self):
        name = "16"
        f = inputFileBase + name + ".txt"
        g = Graph(GraphType.WEIGHTED_DIRECTED)
        g.build_graph(f)
        g.dump(name)
        file = outputFileBase + name + ".dot"
        g.write_dot(file)
        Source(read_dot_file(name))
        assert g.get_numV() == 5
        assert g.get_numE() == 6

    def _DAG(self):
        name = "cat"
        f = inputFileBase + name + ".txt"
        g = Graph(GraphType.DIRECTED)
        g.build_graph(f)
        g.dump(name)
        # g.show_graph()
        file = outputFileBase + name + ".dot"
        g.write_dot(file)
        Source(read_dot_file(name))
        assert g.get_numV() == 6
        assert g.get_numE() == 7

    def _loop(self):
        name = "loopparallel"
        f = inputFileBase + name + ".txt"
        g = Graph(GraphType.WEIGHTED_DIRECTED)
        g.build_graph(f)
        g.dump(name)
        # g.show_graph()
        file = outputFileBase + name + ".dot"
        g.write_dot(file)
        Source(read_dot_file(name))
        assert g.get_numV() == 4
        assert g.get_numE() == 3

    def _test(self):
        self._u1()
        self._uw1()
        self._d1()
        self._dw1()
        self._DAG()
        self._loop()


# Graph tester

In [16]:
############################################################
# GraphTest.py
# Author: Jagadeesh Vasudevamurthy
# Copyright: Jagadeesh Vasudevamurthy 2023
###########################################################

class GraphTest:
    def __init__(self):
        pass

    def RepresentationTest(self):
        t = GraphRepresentationTest()

    def DFS(self):
        t = GraphDfsTest()

    def BFS(self):
        t = GraphBfsTest()

    def Dijkstra(self):
        t = GraphGraphDijkstraTest()

# Main

In [17]:
############################################################
# main
# YOU CANNOT CHANGE ANYTHING BELOW
###########################################################
def main():
    print(sys.version)
    t = GraphTest()
    a = [0, 0, 0, 1]
    if a[0]:
        t.RepresentationTest()
        print("test_graph_representation Passed")
    if a[1]:
        t.DFS()
        print("DFS Passed")
    if a[2]:
        t.BFS() #must fix. Not  completely coded
        print("BFS Passed")
    if a[3]:
        t.Dijkstra()
        print("Dijkstra Passed. You are genius if all tests are passed")
        print("Can you rate me at: https://www.linkedin.com/in/jagadeesh-vasudevamurthy-6796591/")


# call Main

In [18]:
############################################################
# start up
###########################################################
if (__name__  == '__main__'):
    main()

3.9.6 (default, Feb  3 2024, 15:58:27) 
[Clang 15.0.0 (clang-1500.3.9.4)]
-------------  7  ------------
Costs from 0: [0, 5.0, 3.0, 9.0, 13.0, 8.0, 7.0]
Expected cost =  [0.0, 5.0, 3.0, 9.0, 13.0, 8.0, 7.0]
Your answer    =  [0, 5.0, 3.0, 9.0, 13.0, 8.0, 7.0]
Failed  0
-------------  loopparallel  ------------
NOT READ LINE 1 1 3

Costs from s: [0, 1.0, 6.0, 7.0]
Expected cost =  [0.0, 1.0, 6.0, 7.0]
Your answer    =  [0, 1.0, 6.0, 7.0]
Failed  0
-------------  17  ------------
Costs from A: [2.0, 7.0, 5.0, 1.0, 3.0, 7.0, 0]
Expected cost =  [2.0, 7.0, 5.0, 1.0, 3.0, 7.0, 0.0]
Your answer    =  [2.0, 7.0, 5.0, 1.0, 3.0, 7.0, 0]
Failed  0
-------------  hd1  ------------
NOT READ LINE # num problem 1

NOT READ LINE # nodes 20  edge 54

NOT READ LINE 17

NOT READ LINE [20, 25, 25, 68, 86, 39, 22, 70, 36, 53, 91, 35, 88, 27, 30, 43, 54, 74, 41]
Costs from 17: [20.0, 22.0, 25.0, 27.0, 25.0, 68.0, 86.0, 39.0, 70.0, 36.0, 53.0, 91.0, 35.0, 88.0, 30.0, 43.0, 0, 54.0, 74.0, 41.0]
Expected cos

# Uncomment Only Lines 

In [19]:
# Source(read_dot_file("13"))

In [20]:
# Source(read_dot_file("14"))

In [21]:
# Source(read_dot_file("15"))

In [22]:
# Source(read_dot_file("16"))

In [23]:
# Source(read_dot_file("cat"))

In [24]:
# Source(read_dot_file("loopparallel"))

# DFS TRAVERSAL

In [25]:
# Source(read_dot_file("u1dfs"))

In [26]:
# Source(read_dot_file("1dfs"))

In [27]:
# Source(read_dot_file("udf1dfs"))

In [28]:
# Source(read_dot_file("2dfs"))

In [29]:
# Source(read_dot_file("3dfs"))

In [30]:
# Source(read_dot_file("catdfs"))

In [31]:
# Source(read_dot_file("7dfs"))

ouput copy 


3.9.6 (default, Feb  3 2024, 15:58:27) 
[Clang 15.0.0 (clang-1500.3.9.4)]
-------------  7  ------------
Costs from 0: [0, 5.0, 3.0, 9.0, 13.0, 8.0, 7.0]
Expected cost =  [0.0, 5.0, 3.0, 9.0, 13.0, 8.0, 7.0]
Your answer    =  [0, 5.0, 3.0, 9.0, 13.0, 8.0, 7.0]
Failed  0
-------------  loopparallel  ------------
NOT READ LINE 1 1 3

Costs from s: [0, 1.0, 6.0, 7.0]
Expected cost =  [0.0, 1.0, 6.0, 7.0]
Your answer    =  [0, 1.0, 6.0, 7.0]
Failed  0
-------------  17  ------------
Costs from A: [2.0, 7.0, 5.0, 1.0, 3.0, 7.0, 0]
Expected cost =  [2.0, 7.0, 5.0, 1.0, 3.0, 7.0, 0.0]
Your answer    =  [2.0, 7.0, 5.0, 1.0, 3.0, 7.0, 0]
Failed  0
-------------  hd1  ------------
NOT READ LINE # num problem 1

NOT READ LINE # nodes 20  edge 54

NOT READ LINE 17

NOT READ LINE [20, 25, 25, 68, 86, 39, 22, 70, 36, 53, 91, 35, 88, 27, 30, 43, 54, 74, 41]
Costs from 17: [20.0, 22.0, 25.0, 27.0, 25.0, 68.0, 86.0, 39.0, 70.0, 36.0, 53.0, 91.0, 35.0, 88.0, 30.0, 43.0, 0, 54.0, 74.0, 41.0]
Expected cost =  [20.0, 22.0, 25.0, 27.0, 25.0, 68.0, 86.0, 39.0, 70.0, 36.0, 53.0, 91.0, 35.0, 88.0, 30.0, 43.0, 0.0, 54.0, 74.0, 41.0]
Your answer    =  [20.0, 22.0, 25.0, 27.0, 25.0, 68.0, 86.0, 39.0, 70.0, 36.0, 53.0, 91.0, 35.0, 88.0, 30.0, 43.0, 0, 54.0, 74.0, 41.0]
Failed  0
-------------  hd2  ------------
NOT READ LINE # PROBLEM NUMBER 1

NOT READ LINE #NUN MODE NUM EDGE 78 1231

NOT READ LINE # START CITY 60
Expected cost =  [9.0, 13.0, 8.0, 10.0, 8.0, 5.0, 8.0, 5.0, 12.0, 1.0, 7.0, 15.0, 4.0, 8.0, 9.0, 4.0, 11.0, 1.0, 4.0, 12.0, 9.0, 11.0, 7.0, 9.0, 10.0, 9.0, 7.0, 10.0, 5.0, 10.0, 11.0, 9.0, 1.0, 7.0, 12.0, 6.0, 12.0, 15.0, 10.0, 11.0, 15.0, 6.0, 10.0, 7.0, 9.0, 7.0, 7.0, 14.0, 5.0, 13.0, 8.0, 8.0, 10.0, 7.0, 4.0, 6.0, 3.0, 8.0, 11.0, 11.0, 12.0, 4.0, 9.0, 9.0, 7.0, 7.0, 7.0, 0.0, 13.0, 6.0, 7.0, 8.0, 8.0, 3.0, 5.0, 6.0, 11.0, 5.0]
Your answer    =  [9.0, 13.0, 8.0, 10.0, 8.0, 5.0, 8.0, 5.0, 12.0, 1.0, 7.0, 15.0, 4.0, 8.0, 9.0, 4.0, 11.0, 1.0, 4.0, 12.0, 9.0, 11.0, 7.0, 9.0, 10.0, 9.0, 7.0, 10.0, 5.0, 10.0, 11.0, 9.0, 1.0, 7.0, 12.0, 6.0, 12.0, 15.0, 10.0, 11.0, 15.0, 6.0, 10.0, 7.0, 9.0, 7.0, 7.0, 14.0, 5.0, 13.0, 8.0, 8.0, 10.0, 7.0, 4.0, 6.0, 3.0, 8.0, 11.0, 11.0, 12.0, 4.0, 9.0, 9.0, 7.0, 7.0, 7.0, 0, 13.0, 6.0, 7.0, 8.0, 8.0, 3.0, 5.0, 6.0, 11.0, 5.0]
Failed  0
-------------  hd3  ------------
NOT READ LINE # num problem 1

NOT READ LINE # nodes  edge  94 95

NOT READ LINE # START CITY 85

Expected cost =  [154.0, 98.0, 90.0, 49.0, 186.0, 190.0, 178.0, 114.0, 123.0, -1.0, -1.0, -1.0, 123.0, -1.0, 104.0, -1.0, -1.0, -1.0, 207.0, 134.0, 123.0, 75.0, 155.0, -1.0, 198.0, 68.0, 90.0, 170.0, 135.0, -1.0, 103.0, 145.0, -1.0, 54.0, 111.0, 163.0, 173.0, 115.0, 87.0, 159.0, -1.0, 94.0, 102.0, -1.0, 76.0, 67.0, 167.0, 138.0, 216.0, -1.0, 172.0, 102.0, 212.0, 163.0, 103.0, 112.0, -1.0, 182.0, 145.0, 92.0, -1.0, -1.0, 194.0, -1.0, 182.0, -1.0, 201.0, 96.0, -1.0, 85.0, 121.0, 108.0, 161.0, 130.0, 100.0, 120.0, -1.0, 118.0, 215.0, 92.0, 156.0, 162.0, 163.0, 168.0, 0.0, 71.0, 110.0, -1.0, -1.0, 190.0, 217.0, 100.0, 105.0, 178.0]
Your answer    =  [154.0, 98.0, 90.0, 49.0, 186.0, 190.0, 178.0, 114.0, 123.0, -1, -1, -1, 123.0, -1, 104.0, -1, -1, -1, 207.0, 134.0, 123.0, 75.0, 155.0, -1, 198.0, 68.0, 90.0, 170.0, 135.0, -1, 103.0, 145.0, -1, 54.0, 111.0, 163.0, 173.0, 115.0, 87.0, 159.0, -1, 94.0, 102.0, -1, 76.0, 67.0, 167.0, 138.0, 216.0, -1, 172.0, 102.0, 212.0, 163.0, 103.0, 112.0, -1, 182.0, 145.0, 92.0, -1, -1, 194.0, -1, 182.0, -1, 201.0, 96.0, -1, 85.0, 121.0, 108.0, 161.0, 130.0, 100.0, 120.0, -1, 118.0, 215.0, 92.0, 156.0, 162.0, 163.0, 168.0, 0, 71.0, 110.0, -1, -1, 190.0, 217.0, 100.0, 105.0, 178.0]
Failed  0
-------------  g1  ------------
NOT READ LINE 

NOT READ LINE 

NOT READ LINE 

NOT READ LINE 

Costs from P: [0, 1.0, 4.0, 4.0, 2.0, 3.0]
Expected cost =  [0.0, 1.0, 4.0, 4.0, 2.0, 3.0]
Your answer    =  [0, 1.0, 4.0, 4.0, 2.0, 3.0]
Failed  0
-------------  hd4  ------------
NOT READ LINE # test case 1

NOT READ LINE # nun node 68  numedge 462

NOT READ LINE # starting city 49

NOT READ LINE 

Expected cost =  [13.0, 20.0, 30.0, 17.0, 17.0, 34.0, 33.0, 16.0, 16.0, 9.0, 11.0, 31.0, 21.0, 34.0, 19.0, 14.0, 23.0, 20.0, 42.0, 21.0, 15.0, 19.0, 24.0, 15.0, 34.0, 42.0, 27.0, 0, 10.0, 15.0, 16.0, 18.0, 19.0, 18.0, 14.0, 21.0, 21.0, 28.0, 21.0, 19.0, 45.0, 26.0, 27.0, 20.0, 16.0, 23.0, 28.0, 22.0, 27.0, 18.0, 22.0, 21.0, 11.0, 18.0, 22.0, 14.0, 26.0, 27.0, 27.0, 17.0, 24.0, 17.0, 17.0, 22.0, 12.0, 21.0, 10.0, 25.0]
Your answer    =  [13.0, 20.0, 30.0, 17.0, 17.0, 34.0, 33.0, 16.0, 16.0, 9.0, 11.0, 31.0, 21.0, 34.0, 19.0, 14.0, 23.0, 20.0, 42.0, 21.0, 15.0, 19.0, 24.0, 15.0, 34.0, 42.0, 27.0, 0, 10.0, 15.0, 16.0, 18.0, 19.0, 18.0, 14.0, 21.0, 21.0, 28.0, 21.0, 19.0, 45.0, 26.0, 27.0, 20.0, 16.0, 23.0, 28.0, 22.0, 27.0, 18.0, 22.0, 21.0, 11.0, 18.0, 22.0, 14.0, 26.0, 27.0, 27.0, 17.0, 24.0, 17.0, 17.0, 22.0, 12.0, 21.0, 10.0, 25.0]
Failed  0
-------------  hd5  ------------
NOT READ LINE #Nun Mode 28  numedge 242

NOT READ LINE # atrating city 1

Expected cost =  [0, 8.0, 3.0, 11.0, 6.0, 8.0, 7.0, 6.0, 12.0, 12.0, 10.0, 9.0, 18.0, 12.0, 4.0, 8.0, 1.0, 3.0, 1.0, 2.0, 10.0, 8.0, 5.0, 6.0, 9.0, 17.0, 11.0, 8.0]
Your answer    =  [0, 8.0, 3.0, 11.0, 6.0, 8.0, 7.0, 6.0, 12.0, 12.0, 10.0, 9.0, 18.0, 12.0, 4.0, 8.0, 1.0, 3.0, 1.0, 2.0, 10.0, 8.0, 5.0, 6.0, 9.0, 17.0, 11.0, 8.0]
Failed  0
-------------  hd6  ------------
NOT READ LINE # NUM NODE NUMEDGE 88 2824

NOT READ LINE # starting city 1

Expected cost =  [0, 6.0, 3.0, 3.0, 4.0, 3.0, 5.0, 3.0, 3.0, 4.0, 6.0, 5.0, 5.0, 4.0, 4.0, 1.0, 4.0, 5.0, 7.0, 6.0, 6.0, 4.0, 4.0, 4.0, 1.0, 4.0, 5.0, 7.0, 5.0, 2.0, 4.0, 5.0, 5.0, 3.0, 5.0, 6.0, 5.0, 4.0, 5.0, 3.0, 5.0, 5.0, 6.0, 4.0, 5.0, 5.0, 3.0, 6.0, 6.0, 5.0, 3.0, 6.0, 7.0, 5.0, 2.0, 6.0, 6.0, 4.0, 4.0, 3.0, 4.0, 5.0, 3.0, 6.0, 5.0, 3.0, 3.0, 4.0, 7.0, 2.0, 4.0, 6.0, 5.0, 2.0, 2.0, 6.0, 4.0, 6.0, 4.0, 5.0, 3.0, 4.0, 4.0, 4.0, 2.0, 1.0, 3.0, 3.0]
Your answer    =  [0, 6.0, 3.0, 3.0, 4.0, 3.0, 5.0, 3.0, 3.0, 4.0, 6.0, 5.0, 5.0, 4.0, 4.0, 1.0, 4.0, 5.0, 7.0, 6.0, 6.0, 4.0, 4.0, 4.0, 1.0, 4.0, 5.0, 7.0, 5.0, 2.0, 4.0, 5.0, 5.0, 3.0, 5.0, 6.0, 5.0, 4.0, 5.0, 3.0, 5.0, 5.0, 6.0, 4.0, 5.0, 5.0, 3.0, 6.0, 6.0, 5.0, 3.0, 6.0, 7.0, 5.0, 2.0, 6.0, 6.0, 4.0, 4.0, 3.0, 4.0, 5.0, 3.0, 6.0, 5.0, 3.0, 3.0, 4.0, 7.0, 2.0, 4.0, 6.0, 5.0, 2.0, 2.0, 6.0, 4.0, 6.0, 4.0, 5.0, 3.0, 4.0, 4.0, 4.0, 2.0, 1.0, 3.0, 3.0]
Failed  0
Dijkstra Passed. You are genius if all tests are passed
Can you rate me at: https://www.linkedin.com/in/jagadeesh-vasudevamurthy-6796591/